# Data Processing for *Mutual Interactors* implementation

**Bradley Buchner**

In this notebook is code for processing, cleaning, and transforming various datasets required to build and train a *Milieu* (*Mutual Interactors*) model for the discovery of phenotype-protein associations in the roundworm *C. elegans*.

### External Files

Due to size constraints, the following files are not stored in the GitHub repo. Download them manually from the links below and place them in the appropriate folder.

| File | Destination                  | Link |
|------|------------------------------|------|
| `worm_network_basic_full_signed_all.json` | `data/networks/species_6239` | [Download](https://drive.google.com/file/d/1Z7fIipTuTaxl_cnfBDRdUSo9RzbyUXdl/view?usp=sharing) |

In [1]:
import numpy as np
import pandas as pd
import re
import os
import json
import pickle

In [2]:
print("Old wd:", os.getcwd())
parent_dir = os.path.dirname(os.getcwd())
os.chdir(parent_dir)

wd = os.getcwd()
print("New wd:", wd)

Old wd: /Users/bradleybuchner/Desktop/Grad School/Research/Aging Project/milieu/notebooks
New wd: /Users/bradleybuchner/Desktop/Grad School/Research/Aging Project/milieu


In [3]:
path_to_worm_data = os.path.join(wd, "data")
path_to_worm_network = os.path.join(path_to_worm_data, "networks/species_6239/wormbase")
path_to_worm_associations = os.path.join(path_to_worm_data, "associations/wormbase")

### Load the network JSON file

In [4]:
# === Load in the full interaction network ===
# statement_list = "all"
#
# if statement_list == "wormbase":
#     path_to_network_json = os.path.join(path_to_worm_network, "worm_network_basic_full_signed_wb.json")
# elif statement_list == "alliance":
#     path_to_network_json = os.path.join(path_to_worm_network, "worm_network_basic_full_signed.json")
# else:
#     path_to_network_json = os.path.join(path_to_worm_network, "worm_network_basic_full_signed_all.json")

path_to_network_json = os.path.join(path_to_worm_network, "worm_network_basic_full_signed_all.json")
with open(path_to_network_json, "r") as f:
    full_network_data = json.load(f)

### Load the csv of all genes associated with aging in *C. elegans*

In [5]:
# === Load in data file containing all genes in C. elegans that are known to be
# associated with aging (i.e., the "determination of adult lifespan") ===
aging_genes_file_path = os.path.join(path_to_worm_data, "protein_attrs/species_6239/c_elegans_lifespan_genes.txt")
col_names = ["source", "bioentity_internal_id", "bioentity_label", "type", "synonym", "taxon"]
aging_genes_go_df = pd.read_csv(aging_genes_file_path, sep="\t", header=None, names=col_names, dtype=str)

aging_genes_go_df
aging_genes_go = set(aging_genes_go_df['bioentity_label'])

aging_genes_go_df["synonym"] = aging_genes_go_df["synonym"].fillna("").astype(str)
aging_gene_synonyms_go = []
for i in range(aging_genes_go_df.shape[0]):
    synonyms = [x.strip() for x in aging_genes_go_df.loc[i, "synonym"].split("|")]
    aging_gene_synonyms_go.extend(synonyms)

aging_gene_synonyms_go = set(aging_gene_synonyms_go)

print(f"Number of genes annotated with the Gene Ontology term 'determination of adult lifespan': {len(aging_genes_go)}")


Number of genes annotated with the Gene Ontology term 'determination of adult lifespan': 257


In [10]:
# # === Create an entrez id-to-symbol mapping dictionary ===
# db_refs = {}
# entrez_gene_info_path = os.path.join(path_to_worm_data, "Caenorhabditis_elegans.gene_info.txt")
#
# entrez_gene_info = pd.read_csv(entrez_gene_info_path, sep="\t")
# entrez_to_symbol_dict = entrez_gene_info.set_index('GeneID')['Symbol'].to_dict()
# symbol_to_entrez_dict = {sym: id for id, sym in entrez_to_symbol_dict.items()}

### Inspect the network and check that aging-related genes are present

In [6]:
# === Create different node sets from the network (full, aging-related, aging-unrelated) ===
full_node_set = set([node['id'] for node in full_network_data['nodes']])
full_edge_set = set([(edge['source'], edge['target'], np.float64(edge['statements'][0].get('interaction_confidence', 1))) for edge in full_network_data['edges']])
print("Full Network:")
print(f"\tNodes: {len(full_node_set)}")
print(f"\tEdges: {len(full_edge_set)} (with each Complex interactions having two edges)")

# Check that aging-related genes are present in the network
print("\nChecking that all aging-related genes are present in the network...")
for i in range(aging_genes_go_df.shape[0]):
    gene_label = aging_genes_go_df.loc[i, "bioentity_label"]
    gene_synonyms = set([x.strip() for x in aging_genes_go_df.loc[i, "synonym"].split("|")])
    if gene_label not in full_node_set and (len(gene_synonyms.intersection(full_node_set)) == 0):
        print(f"\tWarning: {gene_label} is not in the full network")

# Aging node set
aging_node_set = set()
unknown_node_set = set()
for node in full_node_set:
    if node in aging_genes_go or node in aging_gene_synonyms_go:
        aging_node_set.add(node)
    else:
        unknown_node_set.add(node)

print(f"\nTotal aging-related nodes in the network: {len(aging_node_set)}")
print(f"Total aging-unrelated nodes in the network: {len(unknown_node_set)}")

# Note: Some aging-related genes from the Gene Ontology are missing from the network because they have no known genetic interactions

Full Network:
	Nodes: 11513
	Edges: 156588 (with each Complex interactions having two edges)

Checking that all aging-related genes are present in the network...

Total aging-related nodes in the network: 250
Total aging-unrelated nodes in the network: 11263


### Create gene name-to-ID and ID-to-name lookup dictionaries

In [7]:
# === Define a set of node IDs and create a dictionary that maps IDs to names ===
full_node_set_wbid = set()
node_wbid_to_name = {}
for node in full_network_data['nodes']:
    if 'agA_wbid' in node:
        wbid = node['agA_wbid']
    elif 'agB_wbid' in node:
        wbid = node['agB_wbid']

    full_node_set_wbid.add(wbid)
    name = node['id']
    node_wbid_to_name[wbid] = name

node_name_to_wbid = {name: wbid for wbid, name in node_wbid_to_name.items()}

### Inspect node IDs

In [8]:
aging_node_set_wbid = set([node_name_to_wbid.get(node) for node in aging_node_set])
unknown_node_set_wbid = set([node_name_to_wbid.get(node) for node in unknown_node_set])

# === Inspect the sets of node IDs ===
if len(aging_node_set_wbid.intersection(unknown_node_set)) != 0:
    print("Warning: Data leakage detected")
else:
    print("Confirmed: No data leakage detected")
print(f"\nUnique aging-related WB IDs: {len(aging_node_set_wbid)}")
print(f"Unique aging-unrelated WB IDs: {len(unknown_node_set_wbid)}")

Confirmed: No data leakage detected

Unique aging-related WB IDs: 250
Unique aging-unrelated WB IDs: 11263


### Save the directed and undirected network as adjacency lists

In [9]:
# === Define a set of all edges in the network ===
full_edge_set_wbid = {
    (node_name_to_wbid.get(edge['source']), node_name_to_wbid.get(edge['target']), edge['statements'][0].get('interaction_confidence', 1))
    for edge in full_network_data['edges']
}

#### Directed network

In [10]:
# === Define paths for adjacency list and mappings and save them ===
directed_adj_list_path = os.path.join(path_to_worm_network, "worm_network_basic_full_adjacency_directed.txt")

# Create the directories if they do not exist
os.makedirs(os.path.dirname(directed_adj_list_path), exist_ok=True)

# Save the adjacency list
with open(directed_adj_list_path, "w") as f:
    for source, target, confidence in full_edge_set_wbid:
        f.write(f"{source}\t{target}\t{confidence}\n")

print(f"Adjacency list saved to {full_edge_set_wbid}\n")

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



#### Undirected network

In [11]:
# === Filter out redundant edge duplicates in the network (due to Complex statement double-counting) ===
pair_set = set((src, tgt) for src, tgt, _ in full_edge_set_wbid)
filtered_edge_set_wbid = set()
seen_pairs = set()

for src, tgt, conf in full_edge_set_wbid:
    if (tgt, src) in pair_set:
        if (tgt, src) not in seen_pairs:
            filtered_edge_set_wbid.add((src, tgt, conf))
            seen_pairs.add((src, tgt))
    else:
        filtered_edge_set_wbid.add((src, tgt, conf))

print(f"Number of edges pruned: {len(full_edge_set_wbid)-len(filtered_edge_set_wbid)}")
print(f"\nNumber of edges after pruning: {len(filtered_edge_set_wbid)}")

# === Define paths for adjacency list and mappings and save them ===
adj_list_path = os.path.join(path_to_worm_network, "worm_network_basic_full_adjacency.txt")

# Create the directories if they do not exist
os.makedirs(os.path.dirname(adj_list_path), exist_ok=True)

# Save the adjacency list
with open(adj_list_path, "w") as f:
    for source, target, confidence in filtered_edge_set_wbid:
        f.write(f"{source}\t{target}\t{confidence}\n")

print(f"Adjacency list saved to {adj_list_path}\n")

Number of edges pruned: 71763

Number of edges after pruning: 84825
Adjacency list saved to /Users/bradleybuchner/Desktop/Grad School/Research/Aging Project/milieu/data/networks/species_6239/wormbase/worm_network_basic_full_adjacency.txt



### Load metadata and alternate ID info for all *C. elegans* genes

In [10]:
# === Load WormBase gene metadata ===
# Path to the file containing all C. elegans gene entries from WormBase
worm_all_genes_path = os.path.join(path_to_worm_data, "protein_attrs/species_6239/c_elegans_all_genes.txt")
worm_all_genes_cols = ['source', 'bioentity_internal_id', 'bioentity_label', 'synonym', 'database_xref', 'type', 'taxon', 'taxon_label']
worm_all_genes_df = pd.read_csv(worm_all_genes_path, sep="\t", header=None, names=worm_all_genes_cols, dtype=str) # Load the WormBase gene table into a DataFrame

# === Load additional gene ID aliases ===
# Path to the file with alternate identifiers
worm_gene_other_ids_path = os.path.join(path_to_worm_data, "protein_attrs/species_6239/c_elegans.canonical_bioproject.current.geneOtherIDs.txt")
worm_gene_other_ids_cols = ['wormbase_id', 'status', 'sequence', 'name', 'other_name']
worm_gene_other_ids_df = pd.read_csv(worm_gene_other_ids_path, sep="\t", header=None, names=worm_gene_other_ids_cols, dtype=str)

### Create and save WormBase-ID-to-symbols and symbol-to-WormBase-ID lookup dictionaries

In [12]:
# === Build WormBase ID to gene symbols mapping ===
wb_id_to_symbols = {}
for row in worm_all_genes_df.itertuples(index=False):
    if row.source != 'WB':
        continue
    wb_id = row.bioentity_internal_id
    if wb_id in wb_id_to_symbols:
        continue
    symbols = [row.bioentity_label]
    if pd.notna(row.synonym):
        symbols.extend(x.strip() for x in row.synonym.split("|"))
    wb_id_to_symbols[wb_id] = symbols

# === Augment the mapping with additional ID aliases ===
for row in worm_gene_other_ids_df.itertuples(index=False):
    if row.wormbase_id in wb_id_to_symbols:
        continue
    symbols = [value for value in [row.name, row.sequence, row.other_name] if pd.notna(value)]
    wb_id_to_symbols[row.wormbase_id] = symbols

# === Build gene symbol to WormBase ID reverse lookup dictionary ===
symbol_to_wb_id = {}
for wb_id, symbols in wb_id_to_symbols.items():
    for symbol in symbols:
        if symbol and not symbol in symbol_to_wb_id:
            symbol_to_wb_id[symbol] = wb_id

# === Build gene symbol to list of synonyms lookup dictionary ===
from collections import defaultdict

symbol_to_synonyms = defaultdict(set)
for symbol, wb_id in symbol_to_wb_id.items():
    synonyms = wb_id_to_symbols.get(wb_id, [])
    for syn in synonyms:
        if syn != symbol:
            symbol_to_synonyms[symbol].add(syn)

# Convert sets to lists
symbol_to_synonyms = {k: list(v) for k, v in symbol_to_synonyms.items()}

In [15]:
# === Pickle the lookup dictionaries ===
# Save to pickle
with open(os.path.join(path_to_worm_data, "protein_attrs/species_6239/wb_id_to_symbols.pkl"), "wb") as f:
    pickle.dump(wb_id_to_symbols, f)

with open(os.path.join(path_to_worm_data, "protein_attrs/species_6239/symbol_to_wb_id.pkl"), "wb") as f:
    pickle.dump(symbol_to_wb_id, f)

with open(os.path.join(path_to_worm_data, "protein_attrs/species_6239/symbol_to_synonyms.pkl"), "wb") as f:
    pickle.dump(symbol_to_synonyms, f)

In [24]:
unique_genes = set()
duplicate_genes = set()
for node in list(full_node_set):
    unique_genes.add(node)
    synonyms = symbol_to_synonyms.get(node)
    if synonyms:
        for s in synonyms:
            if s in unique_genes:
                duplicate_genes.add(node)
                print(f"WARNING: {node} already present in network ({s})")

print(len(duplicate_genes))

38


### Parse the raw biological process association data

In [25]:
# === Load gene-biological process associations ===

# File path for raw annotations linking genes to biological processes (GO terms) in C. elegans
bp_gene_associations_file_path = os.path.join(path_to_worm_associations, "biological_process_associations_raw.txt")
bp_gene_associations_column_names = ['source', 'bioentity_internal_id', 'bioentity_label', 'qualifier', 'annotation_class', 'annotation_class_label', 'reference', 'evidence_type', 'evidence_with', 'aspect', 'bioentity_name', 'synonym', 'type', 'taxon', 'date', 'assigned_by', 'annotation_extension_class', 'bioentity_isoform']
bp_gene_associations = pd.read_csv(bp_gene_associations_file_path, sep="\t", header=None, names=bp_gene_associations_column_names)

# === Load UniProt-to-WBGene ID mapping ===
up_to_wormbase_df_path = os.path.join(path_to_worm_associations, "uniprotkb_to_wormbase_id.tsv")
up_to_wormbase_column_names = ['uniprotkb_id', 'gene_names', 'wormbase_xref', 'organism']
up_to_wormbase_df = pd.read_csv(up_to_wormbase_df_path, sep="\t", header=None, skiprows=1, names=up_to_wormbase_column_names)

# === Extract WormBase gene IDs from the "wormbase_xref" column ===
def extract_unique_wb_id(df, col_name="wormbase_xref", new_col_name="wormbase_id"):

    def get_unique_wb_id(cell, row_idx):
        if pd.isna(cell):
            return None
        ids = list(sorted(set(re.findall(r'WBGene\d{8}', cell))))
        if len(ids) > 1:
            return None
            # print(f"Warning: Row {row_idx} contains multiple WBGene IDs: {ids}")
        return ids[0] if ids else None

    df[new_col_name] = [get_unique_wb_id(cell, idx) for idx, cell in enumerate(df[col_name])]
    return df

# Add a clean WormBase ID column to the UniProt-to-WormBase mapping
up_to_wormbase_df = extract_unique_wb_id(up_to_wormbase_df)

### Address missing WBGene IDs in association data

In [26]:
# === Build UniProt-to-WBGene-ID mapping dictionary ===
# Only include mappings where both UniProt ID and WormBase ID are valid
up_to_wb_dict = {}
for row in up_to_wormbase_df.itertuples(index=False):
    if row.uniprotkb_id and row.wormbase_id and row.uniprotkb_id not in up_to_wb_dict:
        up_to_wb_dict[row.uniprotkb_id] = row.wormbase_id

# === Update bp_gene_associations to use WormBase IDs where missing ===
# Replace any non-WBGene IDs in the 'bioentity_internal_id' column
print("Replacing any non-WBGene IDs with WBGene IDs...\n")
for i in range(len(bp_gene_associations)):
    # Try resolving using UniProt → WormBase dictionary
    if 'WBGene' not in bp_gene_associations.loc[i, 'bioentity_internal_id']:
        up_id = bp_gene_associations.loc[i, 'bioentity_internal_id']
        if up_to_wb_dict.get(up_id):
            print(f"{up_id} --> {up_to_wb_dict.get(up_id)} (Using UP ID)")
            bp_gene_associations.loc[i, 'bioentity_internal_id'] = up_to_wb_dict.get(up_id)
        # Try resolving using gene symbol → WormBase dictionary
        elif symbol_to_wb_id.get(bp_gene_associations.loc[i, 'bioentity_label']):
            print(f"{up_id} --> {symbol_to_wb_id.get(bp_gene_associations.loc[i, 'bioentity_label'])} "
                  f"(Using symbol: {bp_gene_associations.loc[i, 'bioentity_label']})")
            bp_gene_associations.loc[i, 'bioentity_internal_id'] = (
                symbol_to_wb_id.get(bp_gene_associations.loc[i, 'bioentity_label']))
        else:
            print(f"Can't get ID for {up_id}")

# === Final check for unmapped/non-WBGene entries ===
for row in bp_gene_associations.itertuples(index=False):
    if 'WBGene' not in row.bioentity_internal_id:
        print("Warning: Invalid gene ID")

Replacing any non-WBGene IDs with WBGene IDs...

Q9N4B8 --> WBGene00022423 (Using symbol: nhr-41)
Q9N4B8 --> WBGene00022423 (Using symbol: nhr-41)
Q9N4B8 --> WBGene00022423 (Using symbol: nhr-41)
Q9U1X4 --> WBGene00014938 (Using symbol: Y62E10A.11)
A0A8S5I9P8 --> WBGene00020914 (Using symbol: sulp-6)
A0A8S5I9P8 --> WBGene00020914 (Using symbol: sulp-6)
A0A8S5I9P8 --> WBGene00020914 (Using symbol: sulp-6)
A0A8S5I9P8 --> WBGene00020914 (Using symbol: sulp-6)
A0A8S5I9P8 --> WBGene00020914 (Using symbol: sulp-6)
A0A8S5I9P8 --> WBGene00020914 (Using symbol: sulp-6)
A0A7I9IA72 --> WBGene00018980 (Using symbol: F56F4.3)
A0A7I9IA72 --> WBGene00018980 (Using symbol: F56F4.3)
A0A7I9IA72 --> WBGene00018980 (Using symbol: F56F4.3)
A0A7R9SUJ3 --> WBGene00013123 (Using symbol: Y52B11A.3)
A0A7R9SUM5 --> WBGene00013015 (Using symbol: Y48E1C.2)
A0A7R9SVK3 --> WBGene00021698 (Using symbol: frl-1)
A0A7R9SVK3 --> WBGene00021698 (Using symbol: frl-1)
A0A7R9SVK3 --> WBGene00021698 (Using symbol: frl-1)
A0A8

### Convert association data into a usable format

In [28]:
# === Group gene associations by biological process ===
# Group by GO ID (`annotation_class`) and GO label (`annotation_class_label`)
# Aggregate associated genes (WBGene IDs) into a sorted, deduplicated list
bp_gene_associations_grouped = (
    bp_gene_associations.groupby(["annotation_class", "annotation_class_label"])["bioentity_internal_id"]
    .apply(lambda x: sorted(set(x.dropna()))) # Drop NaNs and ensure unique, ordered IDs
    .reset_index()
    .rename(columns={
        "annotation_class": "biological_process_id",         # GO term ID
        "annotation_class_label": "biological_process_name", # GO term name
        "bioentity_internal_id": "associated_gene_wb_ids"    # List of WBGene IDs
    })
)

# === Add metadata columns ===
# Add a count column corresponding to the number of associated genes
bp_gene_associations_grouped["count"] = bp_gene_associations_grouped["associated_gene_wb_ids"].apply(len)

# Convert the list of WBGene IDs to a comma-separated string for CSV output
bp_gene_associations_grouped["associated_gene_wb_ids"] = bp_gene_associations_grouped["associated_gene_wb_ids"].apply(lambda ids: ",".join(ids))

# Add a placeholder column for data splits (can be used later for CV or train/test separation)
bp_gene_associations_grouped["splits"] = "none"

display(bp_gene_associations_grouped.head())

print(f"Number of biological processes: {bp_gene_associations_grouped.shape[0]}")

,biological_process_id,biological_process_name,associated_gene_wb_ids,count,splits
0,GO:0000002,mitochondrial genome maintenance,"WBGene00004028,WBGene00011662,WBGene00019800,W...",4,none
1,GO:0000014,single-stranded DNA endodeoxyribonuclease acti...,"WBGene00000787,WBGene00001016,WBGene00003405,W...",4,none
2,GO:0000022,mitotic spindle elongation,"WBGene00002994,WBGene00006381",2,none
3,GO:0000027,ribosomal large subunit assembly,"WBGene00004416,WBGene00007617,WBGene00012692,W...",6,none
4,GO:0000028,ribosomal small subunit assembly,"WBGene00004469,WBGene00004474,WBGene00004483,W...",11,none


Number of biological processes: 4067


### Save the processed association data as a CSV

In [29]:
# === Save the grouped dataset to disk ===
cwd = os.getcwd()
# Write the DataFrame to csv
associations_file_path = os.path.join(path_to_worm_associations, 'biological_process_associations.csv')
bp_gene_associations_grouped.to_csv(associations_file_path, index=False)

print(f"Biological process associations saved to {associations_file_path}\n")

Biological process associations saved to /Users/bradleybuchner/Desktop/Grad School/Research/Aging Project/milieu/data/associations/wormbase/biological_process_associations.csv

